Spark project

In [ ]:
# !pip install praw
# !pip install textblob
# !pip install pyspark

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from textblob import TextBlob

In [ ]:
# Create a SparkContext
sc = SparkContext(appName="RedditStreamingSentimentAnalysis")

In [ ]:
sc.setLogLevel("ERROR")

In [ ]:
# Create a StreamingContext with a batch interval of 10 seconds
ssc = StreamingContext(sc, 1)

In [ ]:
# Create a DStream from the socket
dstream = ssc.socketTextStream("localhost", 8888)

In [ ]:

# Define a function to perform sentiment analysis on each message
def analyze_sentiment(message):
    blob = TextBlob(message)
    polarity = blob.sentiment.polarity
    subjectivity = blob.sentiment.subjectivity
    if polarity > 0:
        sentiment = "positive"
    elif polarity < 0:
        sentiment = "negative"
    else:
        sentiment = "neutral"
    return (message,sentiment)
    # return (sentiment,1)

In [ ]:
# Apply the sentiment analysis function to each message in the DStream
tagged_dstream = dstream.filter(lambda line: len(line) > 0)\
                        .map(lambda message: analyze_sentiment(message))\
                        # .reduceByKey(lambda a, b: a + b)
# .flatMap(lambda line: line.split(" "))\
# .map(lambda line: line.split("|"))\

# Print the results
tagged_dstream.pprint()


In [ ]:
# # For debugging purposes (to check if we receive correctly the data)
# counts = dstream.flatMap(lambda line: line.split(" "))\
#                 .filter(lambda word: len(word) > 0)\
#                 .map(lambda word: (word, 1))\
#                 .reduceByKey(lambda a, b: a + b)
# counts.pprint()

In [ ]:
# Start the streaming context
ssc.start()
# Wait for the streaming to finish
ssc.awaitTermination()

In [ ]:
ssc.stop(stopSparkContext=True, stopGraceFully=True)